In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from transformers import  DistilBertModel,DistilBertTokenizerFast
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything, loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
import pandas as pd
import random
from scripts.dataset import nluDataset
from scripts.model import IC_NER
from scripts.utils import *

C:\Users\khare\miniconda3\envs\ptENV3\lib\site-packages\numpy\__init__.py:138: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


ImportError: numpy.core.multiarray failed to import

In [2]:
def slot_F1(pred,target,id2slots):
    
    pred_list = pred.tolist()
    target_list = target.tolist()
    
    pred_slots , target_slots = [],[]

    for idx_st,t in enumerate(target_list):
        pred_sample,target_sample = [],[]
        for idx_wt,wt in enumerate(t):

            if wt != -100:
                target_sample.append(id2slots[wt])
                pred_sample.append(id2slots[pred_list[idx_st][idx_wt]])

        pred_slots.append(pred_sample)
        target_slots.append(target_sample)
    
    return f1_score( target_slots, pred_slots,mode='strict', scheme=IOB2, average='weighted')

In [3]:
#seed_everything(42)
tb_logger = pl_loggers.TensorBoardLogger('logs/augmented1/01/jointBertEN')
final_slots = pd.read_csv('./data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
idx2slots  = {idx:slots for idx,slots in enumerate(final_slots)}
checkpoint_callback = ModelCheckpoint(dirpath='bin/augmented1/01/EN/v1/',monitor='val_IC_NER_loss', filename='jointBert-{epoch:02d}-{val_loss}')

In [4]:
class NLU_Dataset(pl.LightningDataModule):
    
    def __init__(self, train_dir, val_dir, test_dir,tokenizer, max_len, batch_size):
        
        super().__init__()
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.test_dir = test_dir
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.max_len = max_len

    def setup(self,stage: [str] = None): 
        self.train = nluDataset( self.train_dir, self.tokenizer, self.max_len,2)
        
        self.val = nluDataset( self.val_dir, self.tokenizer, self.max_len,2)
        
        self.test =  nluDataset( self.test_dir, self.tokenizer, self.max_len,2)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [5]:
class jointBert(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.IC_NER = IC_NER('distilbert-base-multilingual-cased')

    def forward(self, input_ids, attention_mask , intent_target, slots_target):
        return self.IC_NER(input_ids, attention_mask , intent_target, slots_target)

    def training_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        
        self.log('train_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_IC_loss', out['ic_loss'], on_step=False, on_epoch=True, logger=True)
        self.log('train_NER_loss', out['ner_loss'], on_step=False, on_epoch=True, logger=True)
        
        return out['joint_loss']
    
    def validation_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        intent_pred, slot_pred = out['intent_pred'], out['slot_pred']
        
        self.log('val_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_IC_loss', out['ic_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_NER_loss', out['ner_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_intent_acc', accuracy(out['intent_pred'],intent_target), on_step=False, on_epoch=True,  logger=True)
        self.log('slot_f1', slot_F1(out['slot_pred'],slots_target,idx2slots), on_step=False, on_epoch=True, logger=True)
        
        
        return out['joint_loss']
    
    def test_step(self,batch,batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        intent_pred, slot_pred = out['intent_pred'], out['slot_pred']
        self.log('test_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('test_IC_loss', out['ic_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('test_NER_loss', out['ner_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('test_intent_acc', accuracy(out['intent_pred'],intent_target), on_step=False, on_epoch=True,  logger=True)
        self.log('test_slot_f1', slot_F1(out['slot_pred'],slots_target,idx2slots), on_step=False, on_epoch=True, logger=True)
        
        return out['joint_loss']
        

    def configure_optimizers(self):
         return torch.optim.AdamW(self.parameters(), lr=3e-5)

In [6]:
dm = NLU_Dataset('./data/multiATIS/split/train/augmented/train_01_EN.tsv','./data/multiATIS/split/valid/augmented/dev_01_EN.tsv', './data/multiATIS/split/test/OOC_test/clean/test_EN.tsv',
                 'distilbert-base-multilingual-cased',56,16)
model = jointBert()

In [6]:
trainer = pl.Trainer(gpus=-1,precision=16,accumulate_grad_batches=4,max_epochs=15, check_val_every_n_epoch=1,logger=tb_logger,callbacks=[checkpoint_callback])

#trainer.fit(model, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [7]:
def cal_mean_stderror(metric):
    var,std_error = 0,0
    mean = sum(metric)/len(metric)
    for m in metric:
        var += (m-mean)**2
    var = (var/(len(metric)-1))**0.5
    std_error = var/((len(metric))**0.5)
    return [round(mean,4),round(std_error,4)]

In [28]:
acc,slotF1 = [],[]
dm1 = NLU_Dataset('./data/multiATIS/split/test/OOC_test/0_10n/test_EN.tsv','./data/multiATIS/split/valid/clean/dev_EN.tsv', 
                  './data/multiATIS/split/test/OOC_test/0_75n/test_EN.tsv',
                 'distilbert-base-multilingual-cased',56,1)
dm1.setup()
test1 = dm1.test_dataloader()

for m in ['./bin/augmented1/010/EN/v1/jointBert-epoch=14-val_loss=0.ckpt',
          './bin/augmented1/010/EN/v2/jointBert-epoch=14-val_loss=0.ckpt']:
    model = jointBert.load_from_checkpoint(checkpoint_path=m,map_location=None)
    model.eval()
    out = trainer.test(model=model,test_dataloaders=test1)
    acc.append(out[0]['test_intent_acc'])
    slotF1.append(out[0]['test_slot_f1'])
    #print(out)
    #print(acc,out,out[0])
print('acc:',cal_mean_stderror(acc),'slotsF1',cal_mean_stderror(slotF1))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IC_NER_loss': 1.2193959951400757,
 'test_IC_loss': 1.3569221496582031,
 'test_NER_loss': 1.0818698406219482,
 'test_intent_acc': 0.7995520830154419,
 'test_slot_f1': 0.662852942943573}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IC_NER_loss': 1.1406915187835693,
 'test_IC_loss': 1.3582725524902344,
 'test_NER_loss': 0.9231106042861938,
 'test_intent_acc': 0.8152295351028442,
 'test_slot_f1': 0.6689974069595337}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IC_NER_loss': 1.2138910293579102,
 'test_IC_loss': 1.311194658279419,
 'test_NER_loss': 1.1165872812271118,
 'test_intent_acc': 0.836506187915802,
 'test_slot_f1': 0.6543858647346497}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IC_NER_loss': 1.2107800245285034,
 'test_IC_loss': 1.4178882837295532,
 'test_NER_loss': 1.0036718845367432,
 'test_intent_acc': 0.8320268988609314,
 'test_slot_f1': 0.6583471894264221}
--------------------------------------------------------------------------------


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_IC_NER_loss': 1.1813247203826904,
 'test_IC_loss': 1.3787857294082642,
 'test_NER_loss': 0.9838634729385376,
 'test_intent_acc': 0.8040313720703125,
 'test_slot_f1': 0.6657431125640869}
--------------------------------------------------------------------------------
acc: [0.8175, 0.0074] slotsF1 [0.6621, 0.0026]
